In [1]:
import pandas as pd

In [4]:
%%bigquery df_equity
select * from equity_prices.rbs_equity where Date > "2015-01-01" order by Date 

In [5]:
df_equity.shape

(1391, 7)

In [6]:
df_equity.dtypes

Date         object
Open         object
High         object
Low          object
Close        object
Adj_Close    object
Volume       object
dtype: object

In [7]:
df_equity['Date'] = pd.to_datetime(df_equity['Date'], errors='coerce')

In [8]:
df_equity.dtypes

Date         datetime64[ns]
Open                 object
High                 object
Low                  object
Close                object
Adj_Close            object
Volume               object
dtype: object

In [9]:
df_equity = df_equity[df_equity['Date']>= '01-01-2018']
df_equity.head()

,Date,Open,High,Low,Close,Adj_Close,Volume
758,2018-01-02,277.200012,279.700012,276.100006,277.299988,241.925720,6870581
759,2018-01-03,277.000000,278.600006,275.500000,275.500000,240.355347,10660425
760,2018-01-04,276.299988,279.399994,275.600006,278.000000,242.536407,12082941
761,2018-01-05,278.000000,279.299988,276.100006,276.299988,241.053284,9925316
762,2018-01-08,276.500000,279.500000,275.799988,278.700012,243.147156,12129375


In [10]:
df_equity.shape

(633, 7)

#### First get a complete data set
Note: our equity prices have missing values (weekends, public holidays).  We should fill in these missing values using the values from the previous day then we will have a set of continuous dates/values ("forward filling").  This will also pre-empt warnings from the prediction methods below.

In [11]:
df_equity.set_index('Date', inplace=True, drop=True)
df_equity.head()

,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2018-01-02,277.200012,279.700012,276.100006,277.299988,241.925720,6870581
2018-01-03,277.000000,278.600006,275.500000,275.500000,240.355347,10660425
2018-01-04,276.299988,279.399994,275.600006,278.000000,242.536407,12082941
2018-01-05,278.000000,279.299988,276.100006,276.299988,241.053284,9925316
2018-01-08,276.500000,279.500000,275.799988,278.700012,243.147156,12129375


In [12]:
df_equity = df_equity.resample('D').ffill().reset_index()
df_equity.head(10)

,Date,Open,High,Low,Close,Adj_Close,Volume
0,2018-01-02,277.200012,279.700012,276.100006,277.299988,241.925720,6870581
1,2018-01-03,277.000000,278.600006,275.500000,275.500000,240.355347,10660425
2,2018-01-04,276.299988,279.399994,275.600006,278.000000,242.536407,12082941
3,2018-01-05,278.000000,279.299988,276.100006,276.299988,241.053284,9925316
4,2018-01-06,278.000000,279.299988,276.100006,276.299988,241.053284,9925316
5,2018-01-07,278.000000,279.299988,276.100006,276.299988,241.053284,9925316
6,2018-01-08,276.500000,279.500000,275.799988,278.700012,243.147156,12129375
7,2018-01-09,276.000000,281.299988,274.700012,280.500000,244.717499,10644412
8,2018-01-10,285.500000,293.600006,283.600006,293.399994,255.971893,27692422
9,2018-01-11,293.000000,297.500000,289.600006,296.799988,258.938171,14965757


Note that the missing dates have been forward filled with the previous day's values.
Now set the index to be the Date because we will want a Series with a DateTime index for methods below.

In [ ]:
ow set the index to be the Date because we will want a Series with a DateTime index for methods below.